In [1]:
%matplotlib inline
import os
from time import sleep

import matplotlib.pyplot as plt
import numpy as np
import qcodes as qc
import time
from qcodes import (
    Measurement,
    experiments,
    initialise_database,
    initialise_or_create_database_at,
    load_by_guid,
    load_by_run_spec,
    load_experiment,
    load_last_experiment,
    load_or_create_experiment,
    new_experiment,
)
from qcodes.dataset.plotting import plot_dataset
from qcodes.logger.logger import start_all_logging
from qcodes.tests.instrument_mocks import DummyInstrument

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\ed261978\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\ed261978\.qcodes\logs\200813-8460-qcodes.log


In [2]:
station = qc.Station()

In [3]:
# A dummy instrument dac with two parameters ch1 and Ch2
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])

# A dummy instrument dmm with two parameters ch1 and Ch2
dmm = DummyInstrument('dmm', gates=['v1', 'v2'])

In [4]:
class dmm_parameter(qc.Parameter):
    def __init__(self, name):
        super().__init__(name)
        self.ed = self.exponential_decay(5, 0.2)
        next(self.ed)

    def get_raw(self):
        """
        This method is automatically wrapped to
        provide a ``get`` method on the parameter instance.
        """
        val = self.ed.send(dac.ch1())
        next(self.ed)
        return val

    @staticmethod
    def exponential_decay(a: float, b: float):
        """
        Yields a*exp(-b*x) where x is put in
        """
        x = 0
        while True:
            x = yield
            yield a*np.exp(-b*x) + 0.02*a*np.random.randn()


dmm.v1 = dmm_parameter('dmm_v1')
dmm.v2 = dmm_parameter('dmm_v2')

In [5]:
initialise_or_create_database_at("livePlot.db")

In [6]:
exp = load_or_create_experiment(experiment_name='dataset_context_manager',
                                sample_name="no sample1")

In [20]:
meas = Measurement(exp=exp, station=station)
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))  # now register the dependent oone
meas.register_parameter(dmm.v2, setpoints=(dac.ch1,))  # now register the dependent oone

meas.write_period = 2

with meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 25):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v),
                             (dmm.v2, get_v*2.))
        time.sleep(0.5)
        print(set_v)

    dataset = datasaver.dataset  # convenient to have for plotting

Starting experimental run with id: 161. 
0.0
1.0416666666666667
2.0833333333333335
3.125
4.166666666666667
5.208333333333334
6.25
7.291666666666667
8.333333333333334
9.375
10.416666666666668
11.458333333333334
12.5
13.541666666666668
14.583333333333334
15.625000000000002
16.666666666666668
17.708333333333336
18.75
19.791666666666668
20.833333333333336
21.875
22.916666666666668
23.958333333333336
25.0


## 2D

In [7]:
meas = Measurement(exp=exp, station=station)
meas.register_parameter(dac.ch1)
meas.register_parameter(dac.ch2)
meas.register_parameter(dmm.v1, setpoints=(dac.ch1, dac.ch2)) 
meas.register_parameter(dmm.v2, setpoints=(dac.ch1, dac.ch2))

meas.write_period = 2

with meas.run() as datasaver:
    
    for set_v1 in np.linspace(0, 25, 5):
        dac.ch1.set(set_v1)
        
        for set_v2 in np.linspace(0, 20, 20):
            dac.ch2.set(set_v2)
            get_v = dmm.v1.get()
            datasaver.add_result((dac.ch1, set_v1),
                                 (dac.ch2, set_v2+set_v1),
                                 
                                 (dmm.v1, get_v),
                                 (dmm.v2, get_v*2))
        
            time.sleep(0.5)
            print(set_v1, set_v2)

    dataset = datasaver.dataset

Starting experimental run with id: 169. 
0.0 0.0
0.0 1.0526315789473684
0.0 2.1052631578947367
0.0 3.1578947368421053
0.0 4.2105263157894735
0.0 5.263157894736842
0.0 6.315789473684211
0.0 7.368421052631579
0.0 8.421052631578947
0.0 9.473684210526315
0.0 10.526315789473683
0.0 11.578947368421051
0.0 12.631578947368421
0.0 13.68421052631579
0.0 14.736842105263158
0.0 15.789473684210526
0.0 16.842105263157894
0.0 17.894736842105264
0.0 18.94736842105263
0.0 20.0
6.25 0.0
6.25 1.0526315789473684
6.25 2.1052631578947367
6.25 3.1578947368421053
6.25 4.2105263157894735
6.25 5.263157894736842
6.25 6.315789473684211
6.25 7.368421052631579
6.25 8.421052631578947
6.25 9.473684210526315
6.25 10.526315789473683
6.25 11.578947368421051
6.25 12.631578947368421
6.25 13.68421052631579
6.25 14.736842105263158
6.25 15.789473684210526
6.25 16.842105263157894
6.25 17.894736842105264
6.25 18.94736842105263
6.25 20.0
12.5 0.0
12.5 1.0526315789473684
12.5 2.1052631578947367
12.5 3.1578947368421053
12.5 4.210

In [ ]:
meas = Measurement(exp=exp, station=station)
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))  # now register the dependent oone
meas.register_parameter(dmm.v2, setpoints=(dac.ch1,))  # now register the dependent oone

meas.write_period = 2

with meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 25):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v),
                             (dmm.v2, get_v*2.))
        time.sleep(0.1)
        print(set_v)

    dataset = datasaver.dataset  # convenient to have for plotting